In [12]:
import wget
import pandas as pd
import json
import datetime
from pandas.io.json import json_normalize

In [48]:
# Baixa o arquivo JS
url = 'http://plataforma.saude.gov.br/novocoronavirus/resources/scripts/'
arquivo = 'database.js'
wget.download(url + arquivo, arquivo)

'database.js'

In [49]:
# Converte Js para Json
base_js = open(arquivo, "rt")
base_json = open("database.json", "wt")

for line in base_js:
	base_json.write(line.replace('var database=', ''))

base_js.close()
base_json.close()

In [44]:
# Exporta o dataframe
df = json.load(open('database.json'))
df = json_normalize(data=df['brazil'], record_path='values',meta=['date','time'])
df[df.uid == 53]

,uid,suspects,refuses,confirmado,deads,local,cases,comments,broadcast,date,time
112,53,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,13/02/2020,16:00
120,53,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,14/02/2020,16:00
128,53,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,15/02/2020,14:00
136,53,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,16/02/2020,12:00
144,53,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,17/02/2020,15:50
160,53,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,19/02/2020,15:50
168,53,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,20/02/2020,15:50
176,53,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,21/02/2020,14:30
184,53,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,22/02/2020,13:40
192,53,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,23/02/2020,12:30


In [45]:
# Insere o nome dos estados
indice = pd.read_csv('indice.csv')
df = pd.merge(df, indice, on='uid', how='left')

In [46]:
# Transforma números em inteiros
df = df.fillna(0)

dados_numericos = ['suspects', 'refuses', 'confirmado', 'deads','cases']

for coluna in dados_numericos:
    df[coluna] = df[coluna].astype(int)

In [47]:
# Exporta a base em CSV com a data da execução no nome do arquivo
agora = str(datetime.date.today()).replace(" ", "").replace(":", "-").replace(".", "-")
arquivo = 'dados/coronabr_' + agora + '.csv'
df.to_csv(arquivo, index = False)